# Deep learning

In [7]:
import torch

from tqdm.auto import tqdm
import numpy as np


DATA_DIR = "data/office"
VAL_SPLIT = .2
RANDOM_SEED = 42
BATCH_SIZE = 16

In [8]:
import sklearn.preprocessing
import imageio.v2 as imageio

import os
from typing import Callable


class ImageDataset(torch.utils.data.Dataset):
    """
    Lazily loads images from a root directory.
    Directory is assumed to be of shape "<root>/<class_name>/<instance_file>".
    """
    def __init__(self, data_dir: str, parser_func: Callable = imageio.imread):
        self.parser_func = parser_func
        self.label_encoder = sklearn.preprocessing.LabelEncoder()
        self.samples = self._load_dataset_paths(data_dir)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = self.parser_func(image_path)
        image_tensor = torch.tensor(image)

        return image_tensor, label

    def _load_dataset_paths(self, data_dir):
        class_names = os.listdir(data_dir)
        self.label_encoder.fit(class_names)

        samples = []
        for class_name in tqdm(class_names):
            class_data_dir = os.path.join(data_dir, class_name)

            for file_name in os.listdir(class_data_dir):
                samples.append(
                    (
                        os.path.join(class_data_dir, file_name),
                        self.label_encoder.transform([class_name])[0],
                    )
                )

        return samples

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler


dataset = ImageDataset(os.path.join(DATA_DIR, "amazon"))
dataset_size = len(dataset)

indices = list(range(dataset_size))
split = int(np.floor(VAL_SPLIT * dataset_size))

np.random.default_rng(RANDOM_SEED).shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=train_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=BATCH_SIZE, sampler=valid_sampler
)

  0%|          | 0/31 [00:00<?, ?it/s]

In [10]:
sample_image, label = dataset[0]
print(sample_image.shape)

torch.Size([300, 300, 3])


In [11]:
print(len(dataset))

2817
